   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 10.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.2 MB/s eta 

In [8]:
import os
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer
from pydantic import BaseModel
from dotenv import load_dotenv
from typing import List

from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [4]:


# Load Mistral 8B
model_name = "mistralai/Mistral-8B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
mistral_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

# --- User & Role Setup (same as before) ---
USER_DB = {...}
ROLE_COLLECTIONS = {...}

app = FastAPI()
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

def authenticate_user(token: str = Depends(oauth2_scheme)):
    username = token
    user = USER_DB.get(username)
    if not user:
        raise HTTPException(status_code=status.HTTP_401_UNAUTHORIZED, detail="Invalid credentials")
    return {"username": username, "role": user["role"]}

class QueryRequest(BaseModel):
    query: str

@app.post("/query")
def query_rag(request: QueryRequest, user=Depends(authenticate_user)):
    role = user["role"]
    allowed_collections = ROLE_COLLECTIONS.get(role, [])
    if not allowed_collections:
        raise HTTPException(status_code=403, detail="No access to any data")

    all_docs = []
    for collection in allowed_collections:
        vectorstore = Chroma(persist_directory=f"./db/{collection}", embedding_function=OpenAIEmbeddings())
        retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
        docs = retriever.get_relevant_documents(request.query)
        all_docs.extend(docs)

    if not all_docs:
        return {"answer": "No relevant data found for your role.", "sources": []}

    context = "\n".join([doc.page_content for doc in all_docs])
    sources = [doc.metadata.get("source", "unknown") for doc in all_docs]

    prompt = (
        f"Answer the following question using only the provided context. Cite sources.\n\n"
        f"Context:\n{context}\n\nQuestion: {request.query}\nAnswer:"
    )
    # Use Mistral 8B for generation
    mistral_output = mistral_pipe(prompt)[0]['generated_text'].split("Answer:")[-1].strip()
    return {"answer": mistral_output, "sources": sources}


ModuleNotFoundError: Module langchain_community.vectorstores not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

In [ ]:
# -*- coding: utf-8 -*-
# RBAC RAG Chatbot: FastAPI backend + Streamlit UI (simplified)

# Install dependencies:
# !pip install fastapi uvicorn streamlit langchain chromadb python-dotenv openai

import os
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer
from pydantic import BaseModel
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from typing import List

load_dotenv()
app = FastAPI()
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# --- Mock user-role mapping ---
USER_DB = {
    "alice": {"password": "finance123", "role": "finance"},
    "bob": {"password": "marketing123", "role": "marketing"},
    "carol": {"password": "hr123", "role": "hr"},
    "dan": {"password": "eng123", "role": "engineering"},
    "eve": {"password": "exec123", "role": "c_level"},
    "frank": {"password": "emp123", "role": "employee"},
}

ROLE_COLLECTIONS = {
    "finance": ["finance_docs"],
    "marketing": ["marketing_docs"],
    "hr": ["hr_docs"],
    "engineering": ["engineering_docs"],
    "c_level": ["finance_docs", "marketing_docs", "hr_docs", "engineering_docs", "general_docs"],
    "employee": ["general_docs"],
}

# --- Authentication & Role Assignment ---
def authenticate_user(token: str = Depends(oauth2_scheme)):
    # In production, use JWT or OAuth2
    username = token  # For demo, token is username
    user = USER_DB.get(username)
    if not user:
        raise HTTPException(status_code=status.HTTP_401_UNAUTHORIZED, detail="Invalid credentials")
    return {"username": username, "role": user["role"]}

# --- Data Retrieval & RAG Pipeline ---
class QueryRequest(BaseModel):
    query: str

@app.post("/query")
def query_rag(request: QueryRequest, user=Depends(authenticate_user)):
    role = user["role"]
    allowed_collections = ROLE_COLLECTIONS.get(role, [])
    if not allowed_collections:
        raise HTTPException(status_code=403, detail="No access to any data")

    # Example: Use Chroma vectorstore for each collection
    all_docs = []
    for collection in allowed_collections:
        vectorstore = Chroma(persist_directory=f"./db/{collection}", embedding_function=OpenAIEmbeddings())
        retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
        docs = retriever.get_relevant_documents(request.query)
        all_docs.extend(docs)

    if not all_docs:
        return {"answer": "No relevant data found for your role.", "sources": []}

    # Build context for LLM
    context = "\n".join([doc.page_content for doc in all_docs])
    sources = [doc.metadata.get("source", "unknown") for doc in all_docs]

    llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=None)
    answer = qa_chain.combine_documents_chain.llm_chain.llm(
        f"Answer the following question using only the provided context. Cite sources.\n\nContext:\n{context}\n\nQuestion: {request.query}\nAnswer:"
    )
    return {"answer": answer, "sources": sources}

# --- Streamlit UI (run separately as streamlit_app.py) ---
# Save this as streamlit_app.py and run: streamlit run streamlit_app.py

"""
import streamlit as st
import requests

st.title("FinSolve RBAC RAG Chatbot")

username = st.text_input("Username")
password = st.text_input("Password", type="password")
query = st.text_input("Ask your question:")

if st.button("Submit"):
    # For demo, token is username
    headers = {"Authorization": f"Bearer {username}"}
    response = requests.post(
        "http://localhost:8000/query",
        json={"query": query},
        headers=headers
    )
    if response.status_code == 200:
        data = response.json()
        st.markdown(f"**Answer:** {data['answer']}")
        st.markdown(f"**Sources:** {', '.join(data['sources'])}")
    else:
        st.error(response.json().get("detail", "Error"))
"""

# To run backend: uvicorn main:app --reload
# To run frontend: streamlit run streamlit_app.py


In [1]:
# Install dependencies
!pip install -q fastapi uvicorn streamlit chromadb transformers accelerate langchain langchain-community langchain-qdrant qdrant-client pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.5 MB/s eta 0:

In [5]:
import os
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer
from pydantic import BaseModel

from dotenv import load_dotenv
from typing import List

from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [9]:
from huggingface_hub import login
login(new_session=True)

In [6]:
# MiniLM for embeddings
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)


/tmp/ipython-input-6-2410786072.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
qdrant_url = "http://localhost:6333"

# Example: dynamically select collection per user role
def get_vectorstore(collection_name):
    return Qdrant.from_existing_collection(
        embeddings=embeddings,
        collection_name=collection_name,
        url=qdrant_url
    )


In [10]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
mistral_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [11]:
USER_DB = {
    "alice": {"password": "finance123", "role": "finance"},
    "bob": {"password": "marketing123", "role": "marketing"},
    "carol": {"password": "hr123", "role": "hr"},
    "dan": {"password": "eng123", "role": "engineering"},
    "eve": {"password": "exec123", "role": "c_level"},
    "frank": {"password": "emp123", "role": "employee"},
}

ROLE_COLLECTIONS = {
    "finance": ["finance_docs"],
    "marketing": ["marketing_docs"],
    "hr": ["hr_docs"],
    "engineering": ["engineering_docs"],
    "c_level": ["finance_docs", "marketing_docs", "hr_docs", "engineering_docs", "general_docs"],
    "employee": ["general_docs"],
}

In [12]:


app = FastAPI()
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

def authenticate_user(token: str = Depends(oauth2_scheme)):
    username = token
    user = USER_DB.get(username)
    if not user:
        raise HTTPException(status_code=status.HTTP_401_UNAUTHORIZED, detail="Invalid credentials")
    return {"username": username, "role": user["role"]}

class QueryRequest(BaseModel):
    query: str


In [13]:
@app.post("/query")
def query_rag(request: QueryRequest, user=Depends(authenticate_user)):
    role = user["role"]
    allowed_collections = ROLE_COLLECTIONS.get(role, [])
    if not allowed_collections:
        raise HTTPException(status_code=403, detail="No access to any data")

    all_docs = []
    for collection in allowed_collections:
        vectorstore = get_vectorstore(collection)
        retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
        docs = retriever.get_relevant_documents(request.query)
        all_docs.extend(docs)

    if not all_docs:
        return {"answer": "No relevant data found for your role.", "sources": []}

    context = "\n".join([doc.page_content for doc in all_docs])
    sources = [doc.metadata.get("source", "unknown") for doc in all_docs]

    prompt = (
        f"Answer the following question using only the provided context. Cite sources.\n\n"
        f"Context:\n{context}\n\nQuestion: {request.query}\nAnswer:"
    )
    mistral_output = mistral_pipe(prompt)[0]['generated_text'].split("Answer:")[-1].strip()
    return {"answer": mistral_output, "sources": sources}


In [ ]:
#!pip install pyngrok


In [14]:
from pyngrok import ngrok
ngrok.set_auth_token("2zlAbCICGR52XgeZBMhf9IqaSjc_3p5gELGQRhrZhpKZ5oLh5")


In [15]:
from pyngrok import ngrok

# Open a tunnel on the port your FastAPI app will run (e.g., 8000)
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url.public_url}")


Public URL: https://e27c0aed551e.ngrok-free.app


In [16]:
import uvicorn
from pyngrok import ngrok

# Expose port 8000
public_url = ngrok.connect(8000)
print(f"Public FastAPI URL: {public_url}")

# Start FastAPI app in background
!uvicorn main:app --host 0.0.0.0 --port 8000 --reload --workers 1 &


Public FastAPI URL: NgrokTunnel: "https://af4b9d30c8a5.ngrok-free.app" -> "http://localhost:8000"
INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [5234] using WatchFiles
ERROR:    Error loading ASGI app. Could not import module "main".


INFO:     Stopping reloader process [5234]


In [19]:
import requests

url = "https://af4b9d30c8a5.ngrok-free.app"
headers = {"Authorization": "Bearer alice"}  # or "bob" depending on your USER_DB
data = {"query": "What is retrieval-augmented generation?"}

response = requests.post(url, json=data, headers=headers)
print(response.json())


JSONDecodeError: Expecting value: line 1 column 1 (char 0)